## 1. Import libraries and data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from keras.layers import Dense, Activation, Dropout, Reshape, Permute
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

C:\Users\vicky\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [2]:
#Create a path to where the data is stored.
path = r'/Users/vicky/OneDrive/文件/careerfoundry/Machine Learning with Python/Datasets'

In [3]:
# Ensures the charts created are displayed in the notebook without the need to "call" them.
%matplotlib inline

In [4]:
#Read in the Cleaned European weather data.
df = pd.read_csv(os.path.join(path,'climate_nodate_cleaned.csv'), index_col=0)

In [5]:
df

,BASEL_cloud_cover,BASEL_global_radiation,BASEL_humidity,BASEL_precipitation,BASEL_pressure,BASEL_sunshine,BASEL_temp_max,BASEL_temp_mean,BASEL_temp_min,BELGRADE_cloud_cover,BELGRADE_global_radiation,BELGRADE_humidity,BELGRADE_precipitation,BELGRADE_pressure,BELGRADE_sunshine,BELGRADE_temp_max,BELGRADE_temp_mean,BELGRADE_temp_min,BUDAPEST_cloud_cover,BUDAPEST_global_radiation,BUDAPEST_humidity,BUDAPEST_precipitation,BUDAPEST_pressure,BUDAPEST_sunshine,BUDAPEST_temp_max,BUDAPEST_temp_mean,BUDAPEST_temp_min,DEBILT_cloud_cover,DEBILT_global_radiation,DEBILT_humidity,DEBILT_precipitation,DEBILT_pressure,DEBILT_sunshine,DEBILT_temp_max,DEBILT_temp_mean,DEBILT_temp_min,DUSSELDORF_cloud_cover,DUSSELDORF_global_radiation,DUSSELDORF_humidity,DUSSELDORF_precipitation,DUSSELDORF_pressure,DUSSELDORF_sunshine,DUSSELDORF_temp_max,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,HEATHROW_cloud_cover,HEATHROW_global_radiation,HEATHROW_humidity,HEATHROW_precipitation,HEATHROW_pressure,HEATHROW_sunshine,HEATHROW_temp_max,HEATHROW_temp_mean,HEATHROW_temp_min,KASSEL_cloud_cover,KASSEL_global_radiation,KASSEL_humidity,KASSEL_precipitation,KASSEL_pressure,KASSEL_sunshine,KASSEL_temp_max,KASSEL_temp_mean,KASSEL_temp_min,LJUBLJANA_cloud_cover,LJUBLJANA_global_radiation,LJUBLJANA_humidity,LJUBLJANA_precipitation,LJUBLJANA_pressure,LJUBLJANA_sunshine,LJUBLJANA_temp_max,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,MAASTRICHT_cloud_cover,MAASTRICHT_global_radiation,MAASTRICHT_humidity,MAASTRICHT_precipitation,MAASTRICHT_pressure,MAASTRICHT_sunshine,MAASTRICHT_temp_max,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MADRID_cloud_cover,MADRID_global_radiation,MADRID_humidity,MADRID_precipitation,MADRID_pressure,MADRID_sunshine,MADRID_temp_max,MADRID_temp_mean,MADRID_temp_min,MUNCHENB_cloud_cover,MUNCHENB_global_radiation,MUNCHENB_humidity,MUNCHENB_precipitation,MUNCHENB_pressure,MUNCHENB_sunshine,MUNCHENB_temp_max,MUNCHENB_temp_mean,MUNCHENB_temp_min,OSLO_cloud_cover,OSLO_global_radiation,OSLO_humidity,OSLO_precipitation,OSLO_pressure,OSLO_sunshine,OSLO_temp_max,OSLO_temp_mean,OSLO_temp_min,SONNBLICK_cloud_cover,SONNBLICK_global_radiation,SONNBLICK_humidity,SONNBLICK_precipitation,SONNBLICK_pressure,SONNBLICK_sunshine,SONNBLICK_temp_max,SONNBLICK_temp_mean,SONNBLICK_temp_min,STOCKHOLM_cloud_cover,STOCKHOLM_global_radiation,STOCKHOLM_humidity,STOCKHOLM_precipitation,STOCKHOLM_pressure,STOCKHOLM_sunshine,STOCKHOLM_temp_max,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,VALENTIA_cloud_cover,VALENTIA_global_radiation,VALENTIA_humidity,VALENTIA_precipitation,VALENTIA_pressure,VALENTIA_sunshine,VALENTIA_temp_max,VALENTIA_temp_mean,VALENTIA_temp_min
0,7,0.32,0.85,0.09,1.0180,0.7,10.9,6.5,0.8,1,0.88,0.81,0.00,1.0195,7.0,7.9,3.7,-0.9,4,0.44,0.67,0.01,1.017,2.3,5.1,2.4,-0.4,7,0.07,0.85,0.25,1.0032,0.0,11.0,9.3,7.4,8,0.12,0.83,0.08,1.0161,0.0,11.5,10.0,7.0,7,0.13,0.91,0.22,1.0010,0.0,8.3,10.6,9.4,8,0.28,0.82,0.48,1.0094,1.6,9.4,7.9,3.9,8,0.20,1.00,0.00,1.0173,0.0,0.5,-0.6,-1.9,7,0.22,0.83,0.32,1.0063,1.0,11.1,9.5,8.5,6,0.53,0.92,0.00,1.0260,1.4,10.8,7.6,4.4,5,0.20,0.67,0.10,1.0304,0.0,10.4,6.9,1.1,8,0.04,0.98,1.14,0.9978,0.0,5.9,4.9,3.8,4,0.48,0.73,0.01,1.0304,2.3,-3.2,-5.9,-8.5,5,0.05,0.98,0.32,1.0114,0.0,4.9,4.2,2.2,5,0.45,0.88,0.34,1.0003,4.7,10.9,8.5,6.0
1,6,0.36,0.84,1.05,1.0180,1.1,10.1,6.1,3.3,6,0.25,0.84,0.00,1.0172,0.0,4.4,2.9,2.2,4,0.18,0.67,0.31,1.017,0.0,3.1,2.3,1.4,8,0.14,0.90,0.06,1.0056,0.1,8.3,7.7,6.4,8,0.18,0.89,0.66,1.0161,0.5,11.0,8.2,7.4,7,0.13,0.98,0.23,1.0051,0.0,10.6,6.1,3.9,6,0.12,0.86,0.27,1.0086,0.0,9.1,7.7,6.8,6,0.56,0.94,0.13,1.0173,3.2,5.5,2.1,-1.3,8,0.17,0.92,1.34,1.0062,0.4,9.9,8.6,7.5,7,0.46,0.86,0.00,1.0254,0.9,12.2,9.8,7.4,6,0.61,0.72,0.30,1.0292,5.1,10.2,6.2,4.2,8,0.04,0.62,0.00,1.0139,0.0,4.9,3.4,2.8,6,0.21,0.97,0.61,1.0292,0.0,-8.5,-9.5,-10.5,5,0.05,0.62,0.06,1.0114,0.0,5.0,4.0,3.0,7,0.25,0.91,0.84,1.0007,0.7,12.1,8.9,5.6
2,8,0.18,0.90,0.30,1.0180,0.0,9.9,8.5,5.1,6,0.67,0.77,0.00,1.0179,3.5,6.4,3.1,-0.5,4,0.30,0.67,0.00,1.017,0.6,5.3,2.7,1.7,6,0.28,0.92,0.01,

In [6]:
#Read in the pleasant weather data.
pleasant = pd.read_csv(os.path.join(path,'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
pleasant

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,20221027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,20221028,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,20221029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,20221030,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
# List of weather stations
weather_stations = [
    'BASEL_',
    'BELGRADE_',
    'BUDAPEST_',
    'DEBILT_',
    'DUSSELDORF_',
    'HEATHROW_',
    'KASSEL_',
    'LJUBLJANA_',
    'MAASTRICHT_',
    'MADRID_',
    'MUNCHENB_',
    'OSLO_',
    'SONNBLICK_',
    'STOCKHOLM_',
    'VALENTIA_']

In [8]:
# Dictionary mapping Each weather station result for confusion matrix
Weather_outcomes = {
    0: 'BASEL_pleasant_weather',
    1: 'BELGRADE_pleasant_weather',
    2: 'BUDAPEST_pleasant_weather',
    3: 'DEBILT_pleasant_weather',
    4: 'DUSSELDORF_pleasant_weather',
    5: 'HEATHROW_pleasant_weather',
    6: 'KASSEL_pleasant_weather',
    7: 'LJUBLJANA_pleasant_weather',
    8: 'MAASTRICHT_pleasant_weather',
    9: 'MADRID_pleasant_weather',
    10: 'MUNCHENB_pleasant_weather',
    11: 'OSLO_pleasant_weather',
    12: 'SONNBLICK_pleasant_weather',
    13: 'STOCKHOLM_pleasant_weather',
    14: 'VALENTIA_pleasant_weather'}

## 2. Reshape the data layers can be fed to the deep learning model 

In [9]:
# Reshape X
X = df.values.reshape(-1, 15, 9)

In [10]:
X.shape

(22950, 15, 9)

In [11]:
y = pleasant.drop(['DATE'], axis=1)
print(y.shape)

(22950, 15)


In [12]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multilabel-indicator'

In [13]:
#Use argmax to get rid of on-hot encoding and supply the numerical value.
y = np.argmax(y, axis = 1)
print(y.shape)
y

(22950,)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [14]:
type_of_target(y)

'multiclass'

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [17]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360,)
(4590, 15, 9) (4590,)


In [18]:
X_train

array([[[  4.  ,   2.26,   0.62, ...,  21.  ,  14.  ,   6.3 ],
        [  4.  ,   2.74,   0.57, ...,  20.6 ,  15.4 ,  10.8 ],
        [  5.  ,   2.18,   0.49, ...,  19.  ,  15.7 ,  12.4 ],
        ...,
        [  7.  ,   2.15,   0.96, ...,  -5.2 ,  -6.9 ,  -8.5 ],
        [  6.  ,   1.34,   0.63, ...,  15.7 ,  11.9 ,   9.8 ],
        [  7.  ,   1.15,   0.93, ...,  12.8 ,  11.1 ,   9.4 ]],

       [[  8.  ,   1.39,   0.8 , ...,  21.2 ,  18.  ,  14.7 ],
        [  8.  ,   0.73,   0.94, ...,  22.  ,  16.7 ,  15.6 ],
        [  8.  ,   0.93,   0.78, ...,  21.6 ,  18.3 ,  15.2 ],
        ...,
        [  8.  ,   1.77,   0.99, ...,   0.1 ,  -1.2 ,  -2.4 ],
        [  2.  ,   3.26,   0.61, ...,  23.8 ,  18.  ,  12.3 ],
        [  7.  ,   1.11,   0.9 , ...,  16.7 ,  15.2 ,  13.7 ]],

       [[  6.  ,   1.18,   0.86, ...,  21.  ,  16.5 ,  14.4 ],
        [  5.  ,   2.45,   0.69, ...,  28.1 ,  22.4 ,  17.2 ],
        [  4.  ,   1.47,   0.67, ...,  23.1 ,  19.1 ,  16.1 ],
        ...,
        [  8

In [19]:
len(X_train[0])

15

In [20]:
len(X_train[0][0])

9

In [21]:
y_train

array([0, 6, 0, ..., 1, 0, 0], dtype=int64)

In [22]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6 #_count_classes(Y_train)
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [28]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'SGD']
    optimizerD = {
        'Adam': Adam(learning_rate=learning_rate),
        'SGD': SGD(learning_rate=learning_rate),
        'RMSprop': RMSprop(learning_rate=learning_rate),
        'Adadelta': Adadelta(learning_rate=learning_rate),
        'Adagrad': Adagrad(learning_rate=learning_rate),
        'Adamax': Adamax(learning_rate=learning_rate),
        'Nadam': Nadam(learning_rate=learning_rate),
        'Ftrl': Ftrl(learning_rate=learning_rate)
    }
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', 'LeakyReLU', 'relu']

    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return model

    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [31]:
start = time.time()
params = {
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation': (0, 9),
    'optimizer': (0, 7),
    'learning_rate': (0.01, 1),
    'batch_size': (200, 1000),
    'epochs': (20, 100),
    'layers1': (1, 3),
    'layers2': (1, 3),
    'normalization': (0, 1),
    'dropout': (0, 1),
    'dropout_rate': (0, 0.3)
}

# Ensure the labels are within the valid range [0, n_classes - 1]
y_train = np.array(y_train)
n_classes = np.max(y_train) + 1
y_train[y_train >= n_classes] = n_classes - 1

# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4)
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/32
16/16 - 3s - 205ms/step - accuracy: 0.3949 - loss: 42.5056
Epoch 2/32
16/16 - 0s - 21ms/step - accuracy: 0.4407 - loss: 43.7275
Epoch 3/32
16/16 - 0s - 22ms/step - accuracy: 0.4542 - loss: 31.1516
Epoch 4/32
16/16 - 0s - 22ms/step - accuracy: 0.5091 - loss: 24.1951
Epoch 5/32
16/16 - 0s - 21ms/step - accuracy: 0.5206 - loss: 14.7544
Epoch 6/32
16/16 - 0s - 22ms/step - accuracy: 0.4975 - loss: 18.6083
Epoch 7/32
16/16 - 0s - 22ms/step - accuracy: 0.5098 - loss: 15.7302
Epoch 8/32
16/16 - 0s - 22ms/step - accuracy: 0.5179 - loss: 17.7034
Epoch 9/32
16/16 - 0s - 22ms/step - accuracy: 0.5394 - loss: 14.2723
Epoch 10/32
16/16 - 0s - 22ms/step - a

ValueError: Input y contains NaN.

In [33]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]
optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]

optimum

{'activation': 'softsign',
 'batch_size': 961,
 'dropout': 0.7319939418114051,
 'dropout_rate': 0.17959754525911098,
 'epochs': 32,
 'kernel': 1.3119890406724053,
 'layers1': 1,
 'layers2': 3,
 'learning_rate': 0.6051038616257767,
 'neurons': 74,
 'normalization': 0.020584494295802447,
 'optimizer': <keras.src.optimizers.ftrl.Ftrl at 0x1f890079150>}

In [36]:
# Define optimizer with the specified learning rate
optimizer = Adadelta(learning_rate=0.6051038616257767)

# Verify the label values in y_train are within the valid range [0, n_classes-1]
y_train = np.array(y_train)
n_classes = 6
y_train = np.clip(y_train, 0, n_classes - 1)

# Parameters
epochs = 32
batch_size = 961
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
layers1 = 1
layers2 = 3
activation = 'softsign'
kernel = 1
neurons = 74
normalization = 0.020584494295802447
dropout = 0.7319939418114051
dropout_rate = 0.17959754525911098

# Build model
model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))
if normalization > 0.5:
    model.add(BatchNormalization())
for i in range(layers1):
    model.add(Dense(neurons, activation=activation))
if dropout > 0.5:
    model.add(Dropout(dropout_rate, seed=123))
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #binary_crossentropy

In [37]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/32
20/20 - 3s - 133ms/step - accuracy: 0.5781 - loss: 1.2188
Epoch 2/32
20/20 - 0s - 21ms/step - accuracy: 0.6415 - loss: 0.9466
Epoch 3/32
20/20 - 0s - 23ms/step - accuracy: 0.6454 - loss: 0.9194
Epoch 4/32
20/20 - 0s - 22ms/step - accuracy: 0.6497 - loss: 0.9029
Epoch 5/32
20/20 - 0s - 21ms/step - accuracy: 0.6614 - loss: 0.8675
Epoch 6/32
20/20 - 0s - 21ms/step - accuracy: 0.6832 - loss: 0.8293
Epoch 7/32
20/20 - 0s - 20ms/step - accuracy: 0.7007 - loss: 0.7947
Epoch 8/32
20/20 - 0s - 20ms/step - accuracy: 0.7139 - loss: 0.7590
Epoch 9/32
20/20 - 0s - 19ms/step - accuracy: 0.7253 - loss: 0.7322
Epoch 10/32
20/20 - 0s - 19ms/step - accuracy: 0.7358 - loss: 0.7098
Epoch 11/32
20/20 - 0s - 20ms/step - accuracy: 0.7449 - loss: 0.6848
Epoch 12/32
20/20 - 0s - 21ms/step - accuracy: 0.7469 - loss: 0.6797
Epoch 13/32
20/20 - 0s - 21ms/step - accuracy: 0.7541 - loss: 0.6629
Epoch 14/32
20/20 - 0s - 20ms/step - accuracy: 0.7631 - loss: 0.6519
Epoch 15/32
20/20 - 0s - 21ms/step - accur

In [50]:
def confusion_matrix(y_true, y_pred):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([weather_stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([weather_stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [51]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Pred         BASEL_  BELGRADE_  BUDAPEST_  DEBILT_  HEATHROW_
True                                                         
BASEL_         2804         64         13        6         56
BELGRADE_       340        504          4        1         26
BUDAPEST_        58         38         37        2         30
DEBILT_          30         10          3       10         12
DUSSELDORF_      13          1          1        1         10
HEATHROW_        23          5          2        1         46
KASSEL_           7          1          1        0          2
LJUBLJANA_       26          6          1        0         20
MAASTRICHT_       5          0          0        0          0
MADRID_         125         21          6        0        199
MUNCHENB_         4          2          0        0          0
OSLO_             5          1          0        0          2
STOCKHOLM_        3          0          0        0          0
VALENTIA_         1          